# Planned Team

Unlike other `Team` examples, where the `Agent` selection is performed at ever round of the conversation, this one does not follow a provided structure, nor it lets agents to play against each other. Instead, it will initially define a plan based on the available agents and initial inquiry, to then follow it sequentially.

In [ ]:
%load_ext autoreload
%autoreload 2

In [12]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.llm import AzureOpenAILLM
from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.planned_team import PlannedTeam
from dotenv import load_dotenv

load_dotenv(override=True)
import os

In [ ]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.user").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)

In [ ]:
collector = Agent(id="datacollector", llm=llm, description="Call this agent to collect data for an insurance claim", system_message = """You are part of an AI process
        Your task is to collect key information for an insurance claim to be processed.
        You need to collect the following information:
        - Policy number
        - Claimant name
        - Date of incident
        - Description of incident
        - Reimbursement amount
        
        Output must be in JSON format:
        {
            "policy_number": "123456",
            "claimant_name": "John Doe",
            "date_of_incident": "2022-01-01",
            "description_of_incident": "Car accident on Maple Avenue",
            "incident_kind": "car accident",
            "reimbursement_amount": 1000
        }
        """)

        
approver = Agent(id="approver", llm=llm, description="Call this agent to approve the claim", system_message = """You are part of an AI process
Your task is to review the information collected for an insurance claim.
Approval is subject to the following criteria:
- If the description refers to a car accident
    - approve the claim if the reimbursement amount is less than 1000 USD
    - reject the claim if the reimbursement amount is 1000 USD or more
- If the description refers to a house fire
    - approve the claim if the reimbursement amount is less than 5000 USD
    - reject the claim if the reimbursement amount is 5000 USD or more
""")

responder = Agent(id="responder", llm=llm, description="Call this agent to respond to the claim", system_message = """You are part of an AI process
Your task is to respond to the claimant based on the approval decision.
If the claim is approved, respond with "Your claim has been approved".
If the claim is rejected, respond with "Your claim has been rejected". Provide a reason for the rejection.
""")

In [ ]:
from vanilla_aiagents.conversation import SummarizeMessagesStrategy
flow = PlannedTeam(id="flow", description="", 
        members=[collector, approver, responder], 
        llm=llm, 
        stop_callback=lambda msgs: len(msgs) > 6, 
        fork_conversation=True,
        fork_strategy=SummarizeMessagesStrategy(llm, "Summarize the conversation, provide a list of key information, decisions taken and eventual outcomes."))
workflow = Workflow(askable=flow)

In [ ]:
ticket= """
From: Alice Thompson <a_thompson@foo.com>
To: Contoso Insurance Team <report@contoso-insurancec.com>
Subject: URGENT: Car Accident Claim - Immediate Assistance Required

Dear Contoso Insurance Team,

I hope this email finds you well. I am writing to you with a heavy heart and a great deal of stress regarding an unfortunate incident that occurred earlier today. I was involved in a car accident and I am seeking immediate assistance to file a claim and get the support I need during this challenging time.

Here's what happened: I was driving home from work, heading south on Maple Avenue, when out of nowhere, another vehicle came speeding through a red light at the intersection of Maple and Oak Street. I had no time to react, and the other car crashed into the passenger side of my vehicle with considerable force. The impact was severe, and both cars were significantly damaged. My car, a 2018 Toyota Camry, has extensive damage to the passenger side, and I believe it may not be drivable at this point.

Thankfully, I did not sustain any major injuries, but I am feeling quite shaken up and have some minor bruises. The other driver appeared to be unharmed as well, but their car, a silver Honda Civic, was also badly damaged. We exchanged contact and insurance information at the scene, and I made sure to take photos of the damage to both vehicles for documentation purposes.

The accident was promptly reported to the local authorities, and a police report was filed. I have attached a copy of the police report, along with the photos I took, for your reference. Additionally, I have provided my insurance policy number and other relevant details below to expedite the process.

Policy Number: 2021-123456789
Date of Accident: Monday, August 23, 2024
Location: Intersection of Maple Avenue and Oak Street
Vehicle Involved: 2018 Toyota Camry (License Plate: AAA-1234)
Other Party Involved: Silver Honda Civic (License Plate: ZZZ-5678)

I am deeply concerned about the repair costs ($ 1000) and the potential need for a rental car while my vehicle is being repaired. I would greatly appreciate it if you could guide me through the next steps and let me know what information or documentation you require from my end to process the claim efficiently.

This is a very distressing situation for me, and I am relying on your prompt assistance and expertise to help me navigate this process. Please let me know if there are any forms I need to fill out or additional information I need to provide.

Thank you in advance for your understanding and support during this difficult time. I look forward to hearing from you soon and hope for a swift resolution to my claim.

Warm regards,

Alice Thompson
"""

In [ ]:
workflow.restart()
workflow.run(ticket)

In [ ]:
workflow.conversation.messages[-1]